In [1]:
##### DNN module

from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.optimizers import RMSprop, Adagrad, Adam, SGD
from keras.layers import Conv2D, Activation, MaxPooling2D, Dropout, Flatten, Dense
from keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l1, l2, l1_l2

#from sklearn.model_selection import train_test_split
#from keras import backend as K
#from keras.applications import ResNet50

Using TensorFlow backend.


In [2]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
##### Common moldule

import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import time
from PIL import Image
from scipy.interpolate import RegularGridInterpolator
import pandas as pd

In [4]:
##### Control Panel

fn_serial   = '019-A-PWK'
batch_size  = 16
num_classes = 5
epochs      = 100
pic_size    = 300
pool_size   = (2, 2)

In [5]:
##### Selection

Conv2D_size = [(3, 3), (5, 5), (7, 7)]
drop_out    = [0., 0.1]
l2_ratio    = [1e-4, 1e-8]
optimizer   = [Adam(lr=0.01),
               Adam(lr=0.001),
               RMSprop(lr=0.01),
               RMSprop(lr=0.001),
               SGD(lr=0.01, momentum=0.95),
               SGD(lr=0.001, momentum=0.95),
              ]

total_item = len(Conv2D_size) * len(drop_out) * len(l2_ratio) * len(optimizer)
print('Total config:', total_item)

Total config: 72


In [6]:
##### Note 

#Conv2D_size = [(3, 3), (5, 5), (7, 7)]
#drop_out    = [0., 0.1]
#l2_ratio    = [1e-4, 1e-8]
#optimizer   = [Adam(lr=0.01),
#               Adam(lr=0.001),
#               RMSprop(lr=0.01),
#               RMSprop(lr=0.001),
#               SGD(lr=0.01, momentum=0.95),
#               SGD(lr=0.001, momentum=0.95),
#              ]

# Good result:
# 0013 0103 0104 0113 0114 1013 1014 1015 1104 1105 1115 2004


In [7]:
def create_model(model_name, Conv2D_size, pic_size, l2_ratio, pool_size, drop_out_ratio):
    model = Sequential()
    model.name=model_name
    model.add(Conv2D(32, Conv2D_size, padding='same', input_shape=(pic_size, pic_size, 3), kernel_regularizer=l2(l2_ratio)))
    model.add(Activation('relu'))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(drop_out_ratio))
    model.add(Conv2D(128, Conv2D_size, padding='same', kernel_regularizer=l2(l2_ratio)))
    model.add(Activation('relu'))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(drop_out_ratio))
    model.add(Conv2D(64, Conv2D_size, padding='same', kernel_regularizer=l2(l2_ratio)))
    model.add(Activation('relu'))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(drop_out_ratio))
    model.add(Conv2D(32, Conv2D_size, padding='same', kernel_regularizer=l2(l2_ratio)))
    model.add(Activation('relu'))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(drop_out_ratio))
    model.add(Flatten())
    model.add(Dense(512, kernel_regularizer=l2(l2_ratio)))
    model.add(Activation('relu'))
    #model.add(BatchNormalization())
    model.add(Dropout(drop_out_ratio))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    return model

In [8]:
def create_compile(optimizer):
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [9]:
def resampleRGI3d(input_mx, resize_to, dtype='float32'):
    # Ref: 10 3-A-7
    # Ver. 2.1
    # input_mx : numpy array, the original target matrix
    # resize_to: list or tuple with 3 int inside
    a, b, c = np.shape(input_mx)
    p, q, r = resize_to
    z_grid = np.linspace(0, p - 1, a)
    y_grid = np.linspace(0, q - 1, b)
    x_grid = np.linspace(0, r - 1, c)
    RGI = RegularGridInterpolator((z_grid, y_grid, x_grid), input_mx)
    z_grid_t2 = np.arange(p)
    y_grid_t2 = np.arange(q)
    x_grid_t2 = np.arange(r)
    meshgrid_para = np.meshgrid(z_grid_t2, y_grid_t2, x_grid_t2)
    RGI_mesh_mx = RGI((meshgrid_para[0], meshgrid_para[1], meshgrid_para[2]))
    RGI_mx = np.transpose(RGI_mesh_mx, axes=[1, 0, 2]).astype(dtype)
    return RGI_mx

def happy_time(start,stop):
    process_time = round(stop - start)
    ss = process_time % 60
    mm = process_time // 60 % 60
    hh = process_time // 3600
    duration = "Process time == {}s == {}H {}m {}s".format(process_time,hh,mm,ss)
    return duration

In [10]:
path_unknown = glob.glob('database/image_data/test/*')

In [11]:
unknown_dict = {}

for j in range(len(path_unknown)):
    serial = os.path.basename(path_unknown[j])[:-4]
    temp_pic = np.asarray(Image.open(path_unknown[j]))
    temp_pic = resampleRGI3d(temp_pic, (pic_size, pic_size, 3))
    temp_shape = np.shape(temp_pic)
    temp_pic = np.expand_dims(temp_pic, axis = 0)
    unknown_dict[serial] = temp_pic

print('Length of dict:', len(list(unknown_dict.keys())))

Length of dict: 2000


In [12]:
##### ImageDataGenerator

augment_generator = ImageDataGenerator(
                                       rotation_range=20,
                                       rescale=1./255,
                                       #featurewise_center=True,
                                       #featurewise_std_normalization=True,
                                       width_shift_range=0.2,
                                       height_shift_range=0.2,
                                       horizontal_flip=True,
                                       #data_format='channels_last'
                                      )

In [13]:
##### Generator for train and validation

train_generator = augment_generator.flow_from_directory('database/image_data/sep_train',
                                                        target_size=(pic_size, pic_size),
                                                        batch_size=batch_size,
                                                        class_mode='categorical')
valid_generator = augment_generator.flow_from_directory('database/image_data/sep_valid',
                                                        target_size=(pic_size, pic_size),
                                                        batch_size=batch_size,
                                                        class_mode='categorical')
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break
train_sample = train_generator.samples
valid_sample = valid_generator.samples

Found 2323 images belonging to 5 classes.
Found 500 images belonging to 5 classes.
data batch shape: (16, 300, 300, 3)
labels batch shape: (16, 5)


In [14]:
loop_start = time.time()
count = 0

for i in range(len(Conv2D_size)): #len(Conv2D_size)
    for j in range(1, 2): #len(drop_out)
        for k in range(0, 1): #len(l2_ratio)
            for m in range(3, 4): #len(optimizer)
                model_name = f'Model_{Conv2D_size[i]}-{drop_out[j]}-{l2_ratio[k]}-{optimizer[m]}'
                model = create_model(model_name, Conv2D_size[i], pic_size, l2_ratio[k], pool_size, drop_out[j])
                create_compile(optimizer[m])
                history = model.fit(train_generator, epochs=epochs, verbose=1,
                                    steps_per_epoch=int(train_sample/batch_size),
                                    validation_data=valid_generator,
                                    validation_steps=int(valid_sample/batch_size))
                id_li = []
                flower_class = []
                for n in range(len(list(unknown_dict.keys()))):
                    serial = list(unknown_dict.keys())[n]
                    pred = model.predict(unknown_dict[serial])[0]
                    id_li.append(serial)
                    flower_class.append(list(pred).index(max(pred)))
                    pred_result_df = pd.DataFrame(columns=['id', 'flower_class'])
                    pred_result_df['id'] = id_li
                    pred_result_df['flower_class'] = flower_class
                pred_result_df['flower_class'].value_counts().sort_index()
                pred_result_df.to_csv(f'model/FS_{fn_serial}/pred_result_FS_{i}-{j}-{k}-{m}.csv', index=False)
                train_acc = model.history.history["accuracy"]
                valid_acc = model.history.history["val_accuracy"]
                np.save(f'model/FS_{fn_serial}/History_{i}-{j}-{k}-{m}.npy', [train_acc, valid_acc])
                plt.figure(figsize=(10,8))
                plt.title(f'Accuracy_{i}-{j}-{k}-{m}')
                plt.ylim([0, 1])
                plt.xlim([0, 50])
                plt.plot(train_acc, label = 'train_acc')
                plt.plot(valid_acc, label = 'valid_acc')
                plt.legend()
                plt.savefig(f'model/FS_{fn_serial}/Accuracy_{i}-{j}-{k}-{m}.png')
                plt.clf()
                plt.show()
                count += 1
                print(f'{count}/{total_item}, Model_{i}-{j}-{k}-{m} done. {happy_time(loop_start, time.time())}')

Epoch 1/100
145/145 [==============================] - 69s 473ms/step - loss: 1.7991 - accuracy: 0.3424 - val_loss: 1.2035 - val_accuracy: 0.4335
Epoch 2/100
145/145 [==============================] - 58s 403ms/step - loss: 1.2479 - accuracy: 0.5059 - val_loss: 0.9350 - val_accuracy: 0.5640
Epoch 3/100
145/145 [==============================] - 57s 396ms/step - loss: 1.1210 - accuracy: 0.5821 - val_loss: 1.0902 - val_accuracy: 0.6281
Epoch 4/100
145/145 [==============================] - 61s 422ms/step - loss: 1.0550 - accuracy: 0.6220 - val_loss: 1.0340 - val_accuracy: 0.6343
Epoch 5/100
145/145 [==============================] - 57s 395ms/step - loss: 0.9941 - accuracy: 0.6376 - val_loss: 0.7537 - val_accuracy: 0.6136
Epoch 6/100
145/145 [==============================] - 57s 391ms/step - loss: 1.0079 - accuracy: 0.6441 - val_loss: 0.9458 - val_accuracy: 0.6508
Epoch 7/100
145/145 [==============================] - 60s 411ms/step - loss: 0.9612 - accuracy: 0.6606 - val_loss: 0.6517 -

Epoch 57/100
145/145 [==============================] - 53s 367ms/step - loss: 0.7298 - accuracy: 0.7777 - val_loss: 0.7980 - val_accuracy: 0.7707
Epoch 58/100
145/145 [==============================] - 53s 363ms/step - loss: 0.7078 - accuracy: 0.7850 - val_loss: 1.1420 - val_accuracy: 0.7438
Epoch 59/100
145/145 [==============================] - 52s 357ms/step - loss: 0.7629 - accuracy: 0.7763 - val_loss: 0.7898 - val_accuracy: 0.7211
Epoch 60/100
145/145 [==============================] - 53s 368ms/step - loss: 0.7105 - accuracy: 0.7942 - val_loss: 1.2697 - val_accuracy: 0.6880
Epoch 61/100
145/145 [==============================] - 52s 360ms/step - loss: 0.7374 - accuracy: 0.7789 - val_loss: 1.2900 - val_accuracy: 0.7231
Epoch 62/100
145/145 [==============================] - 52s 356ms/step - loss: 0.6910 - accuracy: 0.7950 - val_loss: 0.5454 - val_accuracy: 0.7645
Epoch 63/100
145/145 [==============================] - 54s 370ms/step - loss: 0.7391 - accuracy: 0.7789 - val_loss: 0

<Figure size 720x576 with 0 Axes>

1/72, Model_0-1-0-3 done. Process time == 5387s == 1H 29m 47s
Epoch 1/100
145/145 [==============================] - 56s 388ms/step - loss: 5.8528 - accuracy: 0.2752 - val_loss: 1.4319 - val_accuracy: 0.3468
Epoch 2/100
145/145 [==============================] - 58s 400ms/step - loss: 1.3787 - accuracy: 0.4482 - val_loss: 0.9992 - val_accuracy: 0.4628
Epoch 3/100
145/145 [==============================] - 58s 399ms/step - loss: 1.2462 - accuracy: 0.5202 - val_loss: 1.4837 - val_accuracy: 0.4876
Epoch 4/100
145/145 [==============================] - 58s 397ms/step - loss: 1.2182 - accuracy: 0.5605 - val_loss: 1.1136 - val_accuracy: 0.6178
Epoch 5/100
145/145 [==============================] - 57s 396ms/step - loss: 1.0981 - accuracy: 0.5977 - val_loss: 1.1091 - val_accuracy: 0.5847
Epoch 6/100
145/145 [==============================] - 57s 395ms/step - loss: 1.0892 - accuracy: 0.6047 - val_loss: 1.0681 - val_accuracy: 0.6095
Epoch 7/100
145/145 [==============================] - 56s 389

145/145 [==============================] - 54s 371ms/step - loss: 0.7102 - accuracy: 0.7872 - val_loss: 1.5645 - val_accuracy: 0.7583
Epoch 57/100
145/145 [==============================] - 54s 371ms/step - loss: 0.7631 - accuracy: 0.7754 - val_loss: 0.8970 - val_accuracy: 0.7438
Epoch 58/100
145/145 [==============================] - 54s 371ms/step - loss: 0.7155 - accuracy: 0.7855 - val_loss: 1.4503 - val_accuracy: 0.7417
Epoch 59/100
145/145 [==============================] - 54s 372ms/step - loss: 0.7266 - accuracy: 0.7768 - val_loss: 0.5433 - val_accuracy: 0.7087
Epoch 60/100
145/145 [==============================] - 54s 373ms/step - loss: 0.7166 - accuracy: 0.7841 - val_loss: 0.8154 - val_accuracy: 0.7479
Epoch 61/100
145/145 [==============================] - 54s 373ms/step - loss: 0.7258 - accuracy: 0.7720 - val_loss: 1.6037 - val_accuracy: 0.6260
Epoch 62/100
145/145 [==============================] - 54s 371ms/step - loss: 0.7095 - accuracy: 0.7820 - val_loss: 0.9382 - val_a

<Figure size 720x576 with 0 Axes>

2/72, Model_1-1-0-3 done. Process time == 10924s == 3H 2m 4s
Epoch 1/100
145/145 [==============================] - 93s 640ms/step - loss: 2.1001 - accuracy: 0.2648 - val_loss: 1.5607 - val_accuracy: 0.3306
Epoch 2/100
145/145 [==============================] - 92s 635ms/step - loss: 1.4696 - accuracy: 0.3771 - val_loss: 1.0266 - val_accuracy: 0.4194
Epoch 3/100
145/145 [==============================] - 92s 632ms/step - loss: 1.4086 - accuracy: 0.4525 - val_loss: 1.6175 - val_accuracy: 0.5145
Epoch 4/100
145/145 [==============================] - 92s 631ms/step - loss: 1.4322 - accuracy: 0.4824 - val_loss: 1.1719 - val_accuracy: 0.5351
Epoch 5/100
145/145 [==============================] - 91s 629ms/step - loss: 1.2522 - accuracy: 0.5358 - val_loss: 1.2029 - val_accuracy: 0.5289
Epoch 6/100
145/145 [==============================] - 91s 628ms/step - loss: 1.1869 - accuracy: 0.5496 - val_loss: 1.3679 - val_accuracy: 0.3802
Epoch 7/100
145/145 [==============================] - 91s 629m

145/145 [==============================] - 89s 614ms/step - loss: 0.7326 - accuracy: 0.7612 - val_loss: 0.8770 - val_accuracy: 0.7231
Epoch 57/100
145/145 [==============================] - 88s 608ms/step - loss: 0.7149 - accuracy: 0.7720 - val_loss: 0.5627 - val_accuracy: 0.7355
Epoch 58/100
145/145 [==============================] - 89s 611ms/step - loss: 0.7720 - accuracy: 0.7547 - val_loss: 0.6990 - val_accuracy: 0.7376
Epoch 59/100
145/145 [==============================] - 89s 612ms/step - loss: 0.7463 - accuracy: 0.7659 - val_loss: 0.6573 - val_accuracy: 0.6942
Epoch 60/100
145/145 [==============================] - 89s 614ms/step - loss: 0.7201 - accuracy: 0.7547 - val_loss: 0.5241 - val_accuracy: 0.7293
Epoch 61/100
145/145 [==============================] - 88s 608ms/step - loss: 0.7578 - accuracy: 0.7633 - val_loss: 0.5579 - val_accuracy: 0.7252
Epoch 62/100
145/145 [==============================] - 89s 611ms/step - loss: 0.7308 - accuracy: 0.7720 - val_loss: 1.4808 - val_a

<Figure size 720x576 with 0 Axes>

3/72, Model_2-1-0-3 done. Process time == 19941s == 5H 32m 21s
